In [ ]:
import sys, site
print("Python:", sys.version)
print("Executable:", sys.executable)
print("site-packages:", site.getsitepackages())

In [3]:
import os
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\ayish\Lexicon Projects\Hugging Face


In [1]:
import os, sys, torch, transformers, huggingface_hub
from transformers import pipeline

c:\Users\ayish\Lexicon Projects\Hugging Face\hf311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""

print("Python:", sys.version)
print("Executable:", sys.executable)
print("Torch:", torch.__version__, "| CUDA available ->", torch.cuda.is_available())
print("Transformers:", transformers.__version__)
print("HF Hub:", huggingface_hub.__version__)

Python: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
Executable: c:\Users\ayish\Lexicon Projects\Hugging Face\hf311\Scripts\python.exe
Torch: 2.8.0+cpu | CUDA available -> False
Transformers: 4.56.1
HF Hub: 0.35.0


In [4]:
gen = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)
out = gen(
    "Rewrite in simpler English: The committee is currently evaluating potential options for improvement.",
    do_sample=False,  top_p=1.0, max_new_tokens=32
)[0]["generated_text"]
print("Sample output:", out)

Device set to use cpu


Sample output: The committee is currently evaluating potential options for improvement.


In [10]:
import os
import re
from transformers import pipeline
import argparse

In [ ]:
rewrite_pipe= pipeline(
    "text2text-generation", model="google/flan-t5-base", device=-1
)
user_sentence = input("Enter one English sentence to simplify: ").strip()
prompt = (
    "Rewrite the sentence in simpler English. "                 
    "Output exactly one sentence, 8 to 24 words, ending with a period.\n\n"  
    f"Sentence: {user_sentence}"                                
)
result = rewrite_pipe(
    prompt,
    do_sample=False, top_p=1.0, max_new_tokens=32
)

generated = result[0]["generated_text"]

generated_norm = " ".join(generated.strip().split())

word_list = re.findall(r"\b[\w'-]+\b", generated_norm)
word_count= len(word_list)
#check1
ends_with_period = generated_norm.endswith(".") 
#check2
punct_enders = re.findall(r"[.!?]", generated_norm)
one_sentence = (len(punct_enders) == 1) and ends_with_period 
#check3
length_ok = (8 <= word_count <= 24)

print("\n--- Instruction Rewrite ---")
print("Input :", user_sentence)         
print("Output:", generated_norm)

if not ends_with_period:
    print("constraint not satisfied: output must end with a period.")
if not one_sentence:
    print("constraint not satisfied: output must be exactly one sentence (no '?' or '!' or multiple sentences).")
if not length_ok:
    print(f"constraint not satisfied: output must be 8–24 words (got {word_count}).")
    

Device set to use cpu



--- Instruction Rewrite ---
Input : The horse raced past the barn fell," where the initial interpretation is misleading. Other challenging sentences rely on wordplay, like "Time flies like an arrow; fruit flies like a banana," or complex grammatical constructions and long clauses, as seen in sentences with multiple embedded phrases.
Output: Output exactly one sentence, 8 to 24 words, ending with a period. Sentence: The horse raced past the barn fell," where the
constraint not satisfied: output must end with a period.
constraint not satisfied: output must be exactly one sentence (no '?' or '!' or multiple sentences).


In [9]:
sentiment_pipe = pipeline(
    "text-classification",                                   
    model="distilbert-base-uncased-finetuned-sst-2-english", 
    device=-1                                                
)
print("\nPlease enter three sentences for sentiment analysis:") 
s1 = input("Sentence 1: ").strip()                              
s2 = input("Sentence 2: ").strip()                              
s3 = input("Sentence 3: ").strip()
sentences = [s1, s2, s3] 
sentiment_results = sentiment_pipe(sentences, truncation=True)
print("\n===  Sentiment (SST-2) ===")
for i, (text, res) in enumerate(zip(sentences, sentiment_results), start=1):  
    label = res["label"]                                         
    score = float(res["score"])                                 
    print(f"{i}) {text} - {label}  (score: {score:.3f})")  

print(
    "\nNote: SST-2 (the dataset used to fine-tune this model) is a binary sentiment task with only "
    "'POSITIVE' and 'NEGATIVE' labels. The original Stanford Sentiment Treebank includes fine-grained "
    "annotations, but the SST-2 configuration collapses them into two classes, so there is no 'Neutral'. "
    "This means mixed/neutral statements may be forced into positive or negative with lower confidence. "
    "If you need a neutral option, use a model trained for 3+ classes (e.g., a 3-class sentiment model) "
    "or inspect the confidence score: values near 0.5 often indicate ambiguity."
)


Device set to use cpu



Please enter three sentences for sentiment analysis:

===  Sentiment (SST-2) ===
1) The restaurant's food was top-notch, and the service was incredibly friendly! - POSITIVE  (score: 1.000)
2) I was so disappointed by the unprofessional staff and the overall poor quality of the experience. - NEGATIVE  (score: 1.000)
3) I adore this phone's camera, but its battery life is truly terrible - NEGATIVE  (score: 0.999)

Note: SST-2 (the dataset used to fine-tune this model) is a binary sentiment task with only 'POSITIVE' and 'NEGATIVE' labels. The original Stanford Sentiment Treebank includes fine-grained annotations, but the SST-2 configuration collapses them into two classes, so there is no 'Neutral'. This means mixed/neutral statements may be forced into positive or negative with lower confidence. If you need a neutral option, use a model trained for 3+ classes (e.g., a 3-class sentiment model) or inspect the confidence score: values near 0.5 often indicate ambiguity.


In [ ]:
def read_lines(path: str):
    with open(path, "r", encoding="utf-8") as f:
        lines = [ln.strip() for ln in f.readlines()]
    return [ln for ln in lines if ln != ""]

def main():
    parser = argparse.ArgumentParser(description="Step 1: Read inputs from a .txt file")
    parser.add_argument("--infile", default="textfile.txt", help="Path to input .txt (default: textfile.txt)")
    args = parser.parse_args()

    inputs = read_lines(args.infile)
    if not inputs:
        print("No non-empty lines found in infile.")
        return
    print(f"Loaded {len(inputs)} lines from: {args.infile}")
    for idx, text in enumerate(inputs, start=1):
        print(f"{idx:02d}. {text}")

if __name__ == "__main__":
    main()

In [ ]:
text= "        The Eiffel Tower in Paris attracts millions of tourists every year."
t = " ".join(text.strip().split())  
enders = sum(ch in ".!?" for ch in t) 
for ch in t:
    print(ch in ".!?")          

In [ ]:
from transformers import AutoTokenizer
import torch

model_name= "google/flan-t5-base"

max_new_tokens = 32

torch.manual_seed(42)

raw_prompts = [
    # Rewrite (4)
    "Rewrite the sentence in simpler English. End with a period. Sentence: 'Python’s clear syntax helps beginners focus on problem-solving.' Output:",
    "Rewrite the sentence in simpler English. End with a period. Sentence: 'Version control lets teams track changes and work safely together.' Output:",
    "Rewrite the sentence in simpler English. End with a period. Sentence: 'Preprocessing text often includes lowercasing and removing extra spaces.' Output:",
    "Rewrite the sentence in simpler English. End with a period. Sentence: 'Short prompts run faster on CPU because attention scales with length.' Output:",
    # Explain (4)
    "Explain in one sentence what a learning rate does. End with a period.",
    "Explain in one sentence what an API key is used for. End with a period.",
    "Explain in one sentence what a unit test checks. End with a period.",
    "Explain in one sentence what a tokenizer does in NLP. End with a period.",
    # Summarize (4)
    "Summarize in one sentence: 'Pipelines bundle tokenization, the model, and decoding. They are great for quick demos on CPU.' Output:",
    "Summarize in one sentence: 'Batching several prompts can improve throughput. Padding and masks keep shapes compatible.' Output:",
    "Summarize in one sentence: 'Beam search is deterministic and often fluent. Sampling adds creativity but may drift.' Output:",
    "Summarize in one sentence: 'SentencePiece and WordPiece split text into subwords. This keeps vocabulary small and improves coverage.' Output:",
]

batch_prompts = [f"Respond in one sentence: {p}" for p in raw_prompts]
tokenizer = AutoTokenizer.from_pretrained(model_name)
enc_batch = tokenizer(
    batch_prompts,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=128
)


print("Input IDs shape:", enc_batch["input_ids"].shape)  # [B, L]
print("Attention mask shape:", enc_batch["attention_mask"].shape)  # [B, L]
print("Padding token ID:", tokenizer.pad_token_id)

print("attention_mask last row:", enc_batch["attention_mask"][-1].tolist())

c:\Users\ayish\Lexicon Projects\Hugging Face\hf311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Input IDs shape: torch.Size([12, 48])
Attention mask shape: torch.Size([12, 48])
Padding token ID: 0
attention_mask last row: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [2]:
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cpu")
torch.set_grad_enabled(False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
model.eval()

input_ids = enc_batch.input_ids           # shape: (batch, seq_len)
attention_mask = enc_batch.attention_mask # 1 for real tokens, 0 for padding

batch_input_ids = input_ids.to(device)
batch_attention_mask = attention_mask.to(device)

greedy_outputs= model.generate(
    input_ids= batch_input_ids,
    attention_mask= batch_attention_mask,
    max_new_tokens= 32,
    do_sample= False,
    num_beams=1
)

beam_outputs = model.generate(
    input_ids= batch_input_ids,
    attention_mask= batch_attention_mask,
    max_new_tokens= 32,
    do_sample= False,
    num_beams= 5    
)
torch.manual_seed(42)
sampling_outputs = model.generate(
    input_ids= batch_input_ids,
    attention_mask= batch_attention_mask,
    max_new_tokens= 32,
    do_sample= True,
    temperature= 0.8,
    top_p= 0.9
)
greedy_texts = [s.strip() for s in tokenizer.batch_decode(greedy_outputs, skip_special_tokens=True)]
beam_texts   = [s.strip() for s in tokenizer.batch_decode(beam_outputs,   skip_special_tokens=True)]
sample_texts = [s.strip() for s in tokenizer.batch_decode(sampling_outputs, skip_special_tokens=True)]

print("\n=== Generated outputs (per prompt) ===")
for i, (g, b, s) in enumerate(zip(greedy_texts, beam_texts, sample_texts), start=1):
    print(f"{i:02d}. [Greedy] {g}")
    print(f"    [Beam]   {b}")
    print(f"    [Sample] {s}")




=== Generated outputs (per prompt) ===
01. [Greedy] Python’s clear syntax helps beginners focus on problem-solving.
    [Beam]   Python’s clear syntax helps beginners focus on problem-solving.
    [Sample] Python’s clear syntax helps beginners focus on problem-solving.
02. [Greedy] Version control lets teams track changes and work safely together.
    [Beam]   Version control lets teams track changes and work safely together.
    [Sample] Version control lets teams track changes and work safely together.
03. [Greedy] Preprocessing text often includes lowercasing and removing extra spaces.
    [Beam]   Preprocessing text often includes lowercasing and removing extraspaces.
    [Sample] Preprocessing text often includes lowercasing and removing extra spaces.
04. [Greedy] Short prompts run faster on CPU because attention scales with length.
    [Beam]   Short prompts run faster on CPU because attention scales with length.
    [Sample] Short prompts run faster on CPU because attention sca

In [5]:
import re
import math

def is_one_sentence(text):
    groups= re.findall(r'[.!?]+', text)
    return len(groups)==1

def ends_with_period(text):
    return text.strip().endswith(".")

def word_count(text):
    return len(text.strip().split())

def in_word_window(n, low=8, high=24):
    return low <= n <= high

def has_repetition(text):
    toks = text.lower().split()
    bigrams = list(zip(toks, toks[1:]))
    seen = set()
    for bg in bigrams:
        if bg in seen:
            return True
        seen.add(bg)
    return False
def evaluate(outputs):
    
    n = len(outputs)
    wcs = [word_count(t) for t in outputs]
    passes = [
        is_one_sentence(t) and ends_with_period(t) and in_word_window(word_count(t))
        for t in outputs
    ]
    reps = [has_repetition(t) for t in outputs]

    pass_rate = 100.0 * sum(passes) / n if n else 0.0
    avg_wc = sum(wcs) / n if n else 0.0
    std_wc = math.sqrt(sum((w - avg_wc) ** 2 for w in wcs) / n) if n else 0.0
    rep_pct = 100.0 * sum(reps) / n if n else 0.0

    return pass_rate, avg_wc, std_wc, rep_pct


results = {
    "Greedy":   evaluate(greedy_texts),
    "Beam":     evaluate(beam_texts),
    "Sampling": evaluate(sample_texts),
}
print("\n=== Automatic checks — summary per strategy ===")
print(f"{'Strategy':<10} {'Pass%':>6} {'AvgWC':>6} {'StdWC':>6} {'Repeat%':>8}")
for name, (pass_rate, avg_wc, std_wc, rep_pct) in results.items():
    print(f"{name:<10} {pass_rate:6.1f} {avg_wc:6.1f} {std_wc:6.1f} {rep_pct:8.1f}")

def failure_reason(t):
    """Return a concise string explaining why a line failed."""
    reasons = []
    if not is_one_sentence(t):
        reasons.append("more than one sentence")
    if not ends_with_period(t):
        reasons.append("missing final period")
    wc = word_count(t)
    if not in_word_window(wc):
        reasons.append(f"word count {wc} outside 8–24")
    if has_repetition(t):
        reasons.append("repetition detected")
    return ", ".join(reasons) or "passes"

for name, texts in [("Greedy", greedy_texts), ("Beam", beam_texts), ("Sampling", sample_texts)]:
    any_fail = False
    for i, t in enumerate(texts, 1):
        r = failure_reason(t)
        if r != "passes":
            if not any_fail:
                print(f"\n-- Failure reasons for {name} --")
                any_fail = True
            print(f"{i:02d}: {r} | {t}")
    if not any_fail:
        print(f"\n-- Failure reasons for {name} --\n(all passed)")


=== Automatic checks — summary per strategy ===
Strategy    Pass%  AvgWC  StdWC  Repeat%
Greedy      100.0   11.0    2.5      8.3
Beam         66.7   11.2    4.3     16.7
Sampling     91.7   11.2    4.1      8.3

-- Failure reasons for Greedy --
05: repetition detected | A student's rate of learning is the rate at which a student's brain learns a new skill.

-- Failure reasons for Beam --
07: repetition detected | A unit test is a test of a unit of measurement.
08: word count 7 outside 8–24 | A tokenizer is a type of tokenizer.
10: missing final period | You can use a variety of templates to create a simple, easy-to-use template. Choose a template that is easy to use and easy
11: more than one sentence | Beam search is deterministic and often fluent. Sampling adds creativity but may drift.
12: more than one sentence, missing final period, repetition detected | WordPiece and SentencePiece have been renamed WordPiece and SentencePiece

-- Failure reasons for Sampling --
05: word count 7

In [8]:
import time
_ = model.generate(**tokenizer("warmup", return_tensors="pt").to(device),
                   do_sample=False, num_beams=1, max_new_tokens=32)

rep_prompt = batch_prompts[0]  # any representative prompt
t0 = time.perf_counter()
enc1 = tokenizer([rep_prompt], return_tensors="pt", padding=True, truncation=True, max_length=32)
_ = model.generate(input_ids=enc1.input_ids.to(device),
                   attention_mask=enc1.attention_mask.to(device),
                   do_sample=False, num_beams=1,  # greedy for consistency
                   max_new_tokens=32)
t1 = time.perf_counter()
single_sec = t1 - t0

# -------- 2) Small-batch timing (12 prompts) --------
t2 = time.perf_counter()
encb = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=32)
_ = model.generate(input_ids=encb.input_ids.to(device),
                   attention_mask=encb.attention_mask.to(device),
                   do_sample=False, num_beams=1,  # greedy
                   max_new_tokens=32)
t3 = time.perf_counter()
batch_sec = t3 - t2

print("\n=== Timing ===")
print(f"Single input: ~{single_sec:.3f}s")
print(f"Small batch : ~{batch_sec:.3f}s")



=== Timing ===
Single input: ~0.192s
Small batch : ~2.164s
